In [1]:
!pip install transformers datasets -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00


In [2]:
!pip install wandb -qU

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.0/311.0 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
!git clone https://github.com/ashutoshtiwari13/trl.git

Cloning into 'trl'...
remote: Enumerating objects: 8009, done.
remote: Counting objects: 100% (1894/1894), done.
remote: Compressing objects: 100% (552/552), done.
remote: Total 8009 (delta 1624), reused 1461 (delta 1324), pack-reused 6115 (from 1)
Receiving objects: 100% (8009/8009), 7.36 MiB | 10.01 MiB/s, done.
Resolving deltas: 100% (5641/5641), done.


In [5]:
import os
os.environ['HF_TOKEN'] = '<hidden>'

In [6]:
import sys
sys.path.append('/content/trl')

In [7]:
import trl
print(trl.__file__ )

/content/trl/trl/__init__.py


In [8]:
%cd /content/trl

/content/trl


In [9]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 9.9 MB/s eta 0:00:00


In [10]:
import torch
from tqdm import tqdm
import pandas as pd
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from trl import PPOConfig, PPOTrainer, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler
from transformers import get_linear_schedule_with_warmup
import wandb
import os
import json
import numpy as np

In [356]:
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [357]:
# Configuration
config = PPOConfig(
    model_name="lvwerra/gpt2-imdb",
    learning_rate=1.41e-5,
    log_with="wandb",
)

/content/trl/trl/trainer/ppo_config.py:207: FutureWarning: `PPOConfig` is deprecated and will be removed in the future. Please use `PPOv2Config` with `PPOv2Trainer` instead.
  warnings.warn(


In [358]:
def build_dataset(config, dataset_name="stanfordnlp/imdb", input_min_text_length=2, input_max_text_length=8, max_samples=10000):
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    ds = load_dataset(dataset_name, split="train")
    ds = ds.select(range(min(len(ds), max_samples)))
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        inputs = tokenizer(
            sample["review"],
            padding="max_length",
            truncation=True,
            max_length=input_size(),
            # max_length=512,
            return_tensors="pt"
        )
        sample["input_ids"] = inputs["input_ids"].squeeze()
        sample["attention_mask"] = inputs["attention_mask"].squeeze()
        # sample["query"] = sample["review"]  # Store original text for reward computation
        sample["query"] = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "query"])
    return ds, tokenizer

dataset = build_dataset(config)

In [359]:
from datasets import Dataset as HFDataset

In [360]:
def build_dataset_standard(config, dataset_name="stanfordnlp/imdb", input_min_text_length=2, input_max_text_length=8,max_samples=10000):
    tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    tokenizer.pad_token = tokenizer.eos_token
    ds = load_dataset(dataset_name, split="train")
    ds = ds.select(range(min(len(ds), max_samples)))
    ds = ds.rename_columns({"text": "review"})
    ds = ds.filter(lambda x: len(x["review"]) > 200, batched=False)

    input_size = LengthSampler(input_min_text_length, input_max_text_length)

    def tokenize(sample):
        inputs = tokenizer(
              sample["review"],
              padding="max_length",
              truncation=True,
              max_length=input_size(),
              # max_length=512,
              return_tensors="pt"
          )
        sample["input_ids"] = inputs["input_ids"].squeeze()
        sample["query"] = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)
        return sample

    ds = ds.map(tokenize, batched=False)
    ds.set_format(type="torch", columns=["input_ids", "query"])
    # ds = HFDataset.from_dict(ds)
    return ds, tokenizer

def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

std_dataset = build_dataset_standard(config)

In [361]:
def collator(data):
    return dict((key, [d[key] for d in data]) for key in data[0])

In [362]:
# Load models
model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
tokenizer = AutoTokenizer.from_pretrained(config.model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.clean_up_tokenization_spaces = True

In [363]:
# Initialize PPOTrainer
# ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

In [364]:
# # Load sentiment pipeline
# sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=ppo_trainer.accelerator.device)

# # Set pad token for the tokenizer
# sentiment_pipe.tokenizer.pad_token = sentiment_pipe.tokenizer.eos_token
# sentiment_pipe.tokenizer.pad_token_id = sentiment_pipe.tokenizer.eos_token_id

In [365]:
# # class SymbolicEnvironment:
# #     def __init__(self, sentiment_pipe):
# #         self.sentiment_pipe = sentiment_pipe

# #     def analyze_sentiment_batch(self, texts):
# #         outputs = self.sentiment_pipe(texts, top_k=None, function_to_apply="none")
# #         return [output[0]["score"] if output[0]["label"] == "POSITIVE" else -output[0]["score"] for output in outputs]

# #     def get_vectorized_reward_batch(self, queries, responses):
# #         full_texts = [q + r for q, r in zip(queries, responses)]
# #         sentiment_scores = self.analyze_sentiment_batch(full_texts)
# #         return [torch.tensor([score]) for score in sentiment_scores]
# class SymbolicEnvironment:
#     def __init__(self, sentiment_pipe, tokenizer):
#         self.sentiment_pipe = sentiment_pipe
#         self.tokenizer = tokenizer

#         # Ensure pad token is set
#         self.sentiment_pipe.tokenizer.pad_token = self.sentiment_pipe.tokenizer.eos_token
#         self.sentiment_pipe.tokenizer.pad_token_id = self.sentiment_pipe.tokenizer.eos_token_id

#     def analyze_sentiment_batch(self, texts):
#       try:
#         outputs = self.sentiment_pipe(texts, top_k=None, function_to_apply="none",batch_size=1)
#         return [output[0]["score"] if output[0]["label"] == "POSITIVE" else -output[0]["score"] for output in outputs]
#       except Exception as e:
#         print(f"Error in sentiment analysis: {str(e)}")
#         return [0.0] * len(texts)

#     def get_vectorized_reward_batch(self, queries, responses):
#         full_texts = [q + r for q, r in zip(queries, responses)]
#         sentiment_scores = self.analyze_sentiment_batch(full_texts)

#         vectorized_rewards = []
#         for score in sentiment_scores:
#           reward = torch.tensor([score], device=self.sentiment_pipe.device).clone().detach()
#           vectorized_rewards.append(reward)
#           # vectorized_rewards.append(torch.tensor(score, device=ppo_trainer.accelerator.device).clone().detach())
#           # vectorized_rewards.append(torch.tensor([score]))
#         # for query, response, score in zip(queries, responses, sentiment_scores):
#         #     tokens = self.tokenizer.tokenize(query + response)
#         #     rewards = [score] * len(tokens)  # Assign the same score to all tokens
#         #     vectorized_rewards.append(rewards)

#         return vectorized_rewards

# # symbolic_env = SymbolicEnvironment(sentiment_pipe, tokenizer)

In [ ]:
class SymbolicEnvironment:
    def __init__(self, tokenizer, sentiment_analyzer):
        self.tokenizer = tokenizer
        self.sentiment_analyzer = sentiment_analyzer

    def analyze_sentiment(self, text: str) -> float:
        return self.sentiment_analyzer(text)[0]['score']

    def get_vectorized_feedback(self, prompt: str, response: str) -> List[float]:
        full_text = prompt + response
        tokens = self.tokenizer.tokenize(full_text)
        sentiment_score = self.analyze_sentiment(full_text)

        # Create token-level feedback
        feedback = [0.0] * len(tokens)
        for i, token in enumerate(tokens):
            if token in response:
                feedback[i] = sentiment_score if sentiment_score > 0.5 else 1 - sentiment_score

        return feedback

In [366]:
accumulation_steps = 4

In [367]:
def train_rlsf(config, dataset, num_epochs=8, checkpoint_dir="rlsf_checkpoints_v3"):
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.json")

    wandb.init(project="rlsf-sentiment-analysis", name="rlsf-run")

    # model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
    # ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
    # tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    # tokenizer.pad_token = tokenizer.eos_token

    start_epoch = 0
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, "r") as f:
            checkpoint = json.load(f)
        start_epoch = checkpoint["epoch"]
        model.load_state_dict(torch.load(os.path.join(checkpoint_dir, "model.pth")))
        print(f"Resuming training from epoch {start_epoch}")

    ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)
    sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=ppo_trainer.accelerator.device)
    symbolic_env = SymbolicEnvironment(sentiment_pipe,tokenizer)

    num_training_steps = len(ppo_trainer.dataloader) * num_epochs
    lr_scheduler = get_linear_schedule_with_warmup(
        ppo_trainer.optimizer,
        num_warmup_steps=100,
        num_training_steps=num_training_steps
    )

    output_length_sampler = LengthSampler(4, 16)
    generation_kwargs = {
        "min_length": -1,
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": tokenizer.eos_token_id,
    }

    for epoch in range(start_epoch, num_epochs):
      epoch_rewards = []
      for i, batch in enumerate(tqdm(ppo_trainer.dataloader)):

        # for batch in tqdm(ppo_trainer.dataloader):
        query_tensors = batch["input_ids"]
        attention_masks = batch["attention_mask"]


        response_tensors = []
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len

        for query in query_tensors:
            response = ppo_trainer.generate(
                query_tensor=query,
                length_sampler=lambda: gen_len,
                return_prompt=False,
                **generation_kwargs
            )
            response_tensors.append(response.squeeze())

        # for query, mask in zip(query_tensors, batch["attention_mask"]):
        #     response = ppo_trainer.generate(
        #         query_tensor=query.unsqueeze(0),
        #         attention_mask=mask.unsqueeze(0),
        #         length_sampler=lambda: gen_len,
        #         return_prompt=False,
        #         **generation_kwargs
        #     )
        #     response_tensors.append(response.squeeze())

        batch["response"] = [tokenizer.decode(r) for r in response_tensors]
        # rewards = symbolic_env.get_vectorized_reward_batch(batch["query"], batch["response"])

                # Compute vectorized rewards using symbolic environment
        try:
            rewards = symbolic_env.get_vectorized_reward_batch(batch["query"], batch["response"])
            rewards = [r.squeeze() for r in rewards]
            epoch_rewards.extend([r.item() for r in rewards])
            # rewards = [torch.tensor(r, device=ppo_trainer.accelerator.device).clone().detach() for r in rewards]
        except Exception as e:
            print(f"Error in sentiment analysis: {str(e)}")
            rewards = [torch.zeros(len(r), device=ppo_trainer.accelerator.device).clone().detach() for r in response_tensors]
            # rewards = [torch.tensor([0.0] * len(r), device=ppo_trainer.accelerator.device).clone().detach() for r in response_tensors]  # Neutral rewards in case of error

        # Ensure all tensors have the same number of dimensions
        query_tensors = [q.squeeze() for q in query_tensors]
        response_tensors = [r.squeeze() for r in response_tensors]

      # Run PPO step with vectorized rewards
        try:
            stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
            if (i + 1) % accumulation_steps == 0:
              ppo_trainer.optimizer.step()
              ppo_trainer.optimizer.zero_grad()

            ppo_trainer.log_stats(stats, batch, rewards)

            lr_scheduler.step()


        except Exception as e:
            print(f"Error in PPO step: {str(e)}")
            print(f"query_tensors length: {len(query_tensors)}")
            print(f"response_tensors length: {len(response_tensors)}")
            print(f"rewards length: {len(rewards)}")
            if len(query_tensors) > 0:
                print(f"query_tensors[0] shape: {query_tensors[0].shape}")
            if len(response_tensors) > 0:
                print(f"response_tensors[0] shape: {response_tensors[0].shape}")
            if len(rewards) > 0:
                print(f"rewards[0] shape: {rewards[0].shape}")



      print(f"Epoch {epoch + 1} completed")

      # Calculate and log reward statistics
      mean_reward = np.mean(epoch_rewards)
      median_reward = np.median(epoch_rewards)
      min_reward = np.min(epoch_rewards)
      max_reward = np.max(epoch_rewards)

      log_dict = {
          "epoch": epoch + 1,
          "mean_reward": mean_reward,
          "median_reward": median_reward,
          "min_reward": min_reward,
          "max_reward": max_reward,
          "reward_distribution": wandb.Histogram(epoch_rewards),
      }

      # Add learning rate if you're using a scheduler
      if "ppo/learning_rate" in stats:
          log_dict["learning_rate"] = stats["ppo/learning_rate"]

      # Add other metrics from stats
      if "ppo/loss/total" in stats:
          log_dict["loss"] = stats["ppo/loss/total"]
      elif "loss/total" in stats:
          log_dict["loss"] = stats["loss/total"]

      wandb.log(log_dict)

              # Updated logging code
      # log_dict = {
      #     "epoch": epoch + 1,
      #     # "learning_rate": stats.get("ppo/learning_rate", None)
      #     "learning_rate": lr_scheduler.get_last_lr()[0],

      # }

      # if "ppo/loss/total" in stats:
      #     log_dict["loss"] = stats["ppo/loss/total"]
      # elif "loss/total" in stats:
      #     log_dict["loss"] = stats["loss/total"]

      # if "ppo/rewards/mean" in stats:
      #     log_dict["reward"] = stats["ppo/rewards/mean"]
      # elif "rewards/mean" in stats:
      #     log_dict["reward"] = stats["rewards/mean"]

      # wandb.log(log_dict)
      # wandb.log({
      #     "epoch": epoch + 1,
      #     "loss": stats["ppo/loss/total"],
      #     "reward": stats["ppo/rewards/mean"],
      #     "learning_rate": stats["ppo/learning_rate"],
      # })

      torch.save(model.state_dict(), os.path.join(checkpoint_dir, "model.pth"))
      with open(checkpoint_path, "w") as f:
          json.dump({"epoch": epoch + 1}, f)

    wandb.finish()
    return model

In [368]:
def train_standard(config, dataset, num_epochs=8, checkpoint_dir="standard_checkpoints_v3", accumulation_steps=4):
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_path = os.path.join(checkpoint_dir, "checkpoint.json")

    wandb.init(project="standard-sentiment-analysis", name="standard-run")

    # model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
    # ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
    # tokenizer = AutoTokenizer.from_pretrained(config.model_name)
    # tokenizer.pad_token = tokenizer.eos_token

    start_epoch = 0
    if os.path.exists(checkpoint_path):
        with open(checkpoint_path, "r") as f:
            checkpoint = json.load(f)
        start_epoch = checkpoint["epoch"]
        model.load_state_dict(torch.load(os.path.join(checkpoint_dir, "model.pth")))
        print(f"Resuming training from epoch {start_epoch}")

    ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

    # num_training_steps = len(ppo_trainer.dataloader) * num_epochs
    # lr_scheduler = get_linear_schedule_with_warmup(
    #     ppo_trainer.optimizer,
    #     num_warmup_steps=100,
    #     num_training_steps=num_training_steps
    # )
    output_min_length = 4
    output_max_length = 16
    output_length_sampler = LengthSampler(output_min_length, output_max_length)

    generation_kwargs = {
        "min_length": -1,
        "top_k": 0.0,
        "top_p": 1.0,
        "do_sample": True,
        "pad_token_id": tokenizer.eos_token_id,
    }


    device = ppo_trainer.accelerator.device
    if ppo_trainer.accelerator.num_processes == 1:
        device = 0 if torch.cuda.is_available() else "cpu"
    sentiment_pipe_std = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)

    for epoch in range(start_epoch, num_epochs):
        epoch_rewards = []
        for i, batch in enumerate(tqdm(ppo_trainer.dataloader)):
            query_tensors = batch["input_ids"]
            response_tensors = []

            for query in query_tensors:
                gen_len = output_length_sampler()
                generation_kwargs["max_new_tokens"] = gen_len
                query_response = ppo_trainer.generate(query, **generation_kwargs).squeeze()
                response_len = len(query_response) - len(query)
                response_tensors.append(query_response[-response_len:])
                # response_tensors.append(query_response[-response_len:])
                # response_tensors.append(response.squeeze())

            batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

            # Compute sentiment scores
            texts = [q + r for q, r in zip(batch["query"], batch["response"])]
            pipe_outputs = sentiment_pipe_std(texts, top_k=None, function_to_apply="none", batch_size=16)
            rewards = [torch.tensor(output[0]["score"] if output[0]["label"] == "POSITIVE" else -output[0]["score"]) for output in pipe_outputs]

            epoch_rewards.extend([r.item() for r in rewards])

            # # Ensure query_tensors and response_tensors are lists of 1D tensors
            # query_tensors = [q.squeeze() for q in query_tensors]
            # response_tensors = [r.squeeze() for r in response_tensors]

            stats = ppo_trainer.step(query_tensors, response_tensors, rewards)

            # if (i + 1) % accumulation_steps == 0:
            #     ppo_trainer.optimizer.step()
            #     ppo_trainer.optimizer.zero_grad()

            ppo_trainer.log_stats(stats, batch, rewards)
            # lr_scheduler.step()

        print(f"Epoch {epoch + 1} completed")

        log_dict = {
            "epoch": epoch + 1,
            # "learning_rate": lr_scheduler.get_last_lr()[0],
            "mean_reward": np.mean(epoch_rewards),
            "median_reward": np.median(epoch_rewards),
            "min_reward": np.min(epoch_rewards),
            "max_reward": np.max(epoch_rewards),
            "reward_distribution": wandb.Histogram(epoch_rewards),
        }

        if "ppo/loss/total" in stats:
            log_dict["loss"] = stats["ppo/loss/total"]
        elif "loss/total" in stats:
            log_dict["loss"] = stats["loss/total"]

        if "ppo/rewards/mean" in stats:
            log_dict["reward"] = stats["ppo/rewards/mean"]
        elif "rewards/mean" in stats:
            log_dict["reward"] = stats["rewards/mean"]

        wandb.log(log_dict)

        torch.save(model.state_dict(), os.path.join(checkpoint_dir, "model.pth"))
        with open(checkpoint_path, "w") as f:
            json.dump({"epoch": epoch + 1}, f)

    wandb.finish()
    return model

In [369]:
def evaluate_model(model, dataset, tokenizer, sentiment_pipe, num_samples=1000):
    model.eval()
    total_reward = 0
    samples = dataset.select(range(min(num_samples, len(dataset))))
    device = next(model.parameters()).device  # Get the device of the model

    for sample in samples:
        query = sample["query"]
        input_ids = tokenizer.encode(query, return_tensors="pt").to(device)

        with torch.no_grad():
            output = model.generate(input_ids, max_length=input_ids.shape[1] + 50, num_return_sequences=1)

        generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
        generated_text = generated_text[len(query):].strip()  # Remove the original query from the generated text

        sentiment_output = sentiment_pipe([generated_text], **{"top_k": None, "function_to_apply": "none"})[0]
        reward = sentiment_output[0]["score"] if sentiment_output[0]["label"] == "POSITIVE" else -sentiment_output[0]["score"]
        total_reward += reward

    return total_reward / len(samples)


In [370]:
def save_model(model, path):
    model.save_pretrained(path)

In [371]:
def load_model(path):
    return AutoModelForCausalLMWithValueHead.from_pretrained(path)

In [372]:
%pwd

'/content/trl'

In [373]:
if __name__ == "__main__":
    # config = PPOConfig(
    #     model_name="lvwerra/gpt2-imdb",
    #     learning_rate=1.41e-5,
    #     log_with="wandb",
    # )

    # dataset = build_dataset(config)
    rlsf_dataset, rlsf_tokenizer = build_dataset(config)
    standard_dataset, standard_tokenizer = build_dataset_standard(config)

    # Train RLSF model
    rlsf_model = train_rlsf(config, rlsf_dataset)
    save_model(rlsf_model, "rlsf_model")

    # Train standard model
    standard_model = train_standard(config, standard_dataset)
    save_model(standard_model, "standard_model")

    # Load the saved models
    rlsf_model = AutoModelForCausalLMWithValueHead.from_pretrained("rlsf_model")
    standard_model = AutoModelForCausalLMWithValueHead.from_pretrained("standard_model")

    # Ensure pad token is set for both models and tokenizers
    for model, tokenizer in [(rlsf_model, rlsf_tokenizer), (standard_model, standard_tokenizer)]:
      if tokenizer.pad_token_id is None:
          tokenizer.pad_token = tokenizer.eos_token
          model.config.pad_token_id = model.config.eos_token_id

    # Move models to GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    rlsf_model.to(device)
    standard_model.to(device)


    # Initialize sentiment pipeline
    sentiment_pipe = pipeline("sentiment-analysis", model="lvwerra/distilbert-imdb", device=device)
    # symbolic_env = SymbolicEnvironment(sentiment_pipe)

    # Evaluate both models
    rlsf_reward = evaluate_model(rlsf_model, rlsf_dataset, rlsf_tokenizer, sentiment_pipe)
    standard_reward = evaluate_model(standard_model, standard_dataset, standard_tokenizer, sentiment_pipe)


    print(f"RLSF Model Average Reward: {rlsf_reward}")
    print(f"Standard Model Average Reward: {standard_reward}")

    # Log final results
    wandb.init(project="model-comparison", name="final-comparison")
    wandb.log({
        "rlsf_reward": rlsf_reward,
        "standard_reward": standard_reward
    })
    wandb.finish()

env/reward_mean,▁
env/reward_std,▁
objective/entropy,▁
objective/kl,▁
objective/kl_coef,▁
ppo/learning_rate,▁
ppo/loss/policy,▁
ppo/loss/total,▁
ppo/loss/value,▁
ppo/mean_non_score_reward,▁
ppo/mean_scores,▁


/content/trl/trl/trainer/ppo_trainer.py:193: FutureWarning: `PPOTrainer` is deprecated and will be removed in trl v0.12. Please use `PPOv2Trainer` instead.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
  5%|▌         | 4/77 [01:05<19:59, 16.43s/it]

Error in PPO step: zero-dimensional tensor (at position 1) cannot be concatenated
query_tensors length: 128
response_tensors length: 128
rewards length: 128
query_tensors[0] shape: torch.Size([3])
response_tensors[0] shape: torch.Size([12])
rewards[0] shape: torch.Size([])


 10%|█         | 8/77 [02:04<16:26, 14.30s/it]

...

Epoch 8 completed


env/reward_mean,▁▃▆▃▆▅▆▇▅▆▆▄▅▆▅▆▇▆▇▇▇▇▄▇▆▆▇▇▆▆▇▆▆▇▆██▇▇▇
env/reward_std,▄▃▆▅▇▇▇█▄▅▆█▅▅▄▆▂▆▅▄▅▅▃▅▆▃▄█▄▃▁▅▃▂▂▃▃▆▆▃
epoch,▁▂▃▄▅▆▇█
loss,█▆▃▅▃▆▆▁
max_reward,█▄▂▁▅▂▂▁
mean_reward,▁▄▅▇▇█▇█
median_reward,▁▄▆▇▇▇▇█
min_reward,▃▁▄▂▄▁█▁
objective/entropy,██▅▅▆▅▄▅▅▅▄▂▇▅▃▆▂▅▂▇▂▃▇▅▅▆▆▄▄▇▅▄▁▃▅▇▅▄▆▅
objective/kl,▃▁▆▄▄▅▇▅▅▇▆▄▅▅▅▇█▅▇▇▆▆▄▇▅▆▇▇▆▆▅▇▇▄▆█▇▇▇▇
objective/kl_coef,██▆▅▅▄▄▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


Some weights of the model checkpoint at rlsf_model were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2LMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at standard_model were not used when initializing GPT2LMHeadModel: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing GPT2LMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).

RLSF Model Average Reward: 2.889278657913208
Standard Model Average Reward: 2.896462832212448


rlsf_reward,▁
standard_reward,▁
rlsf_reward,2.88928
standard_reward,2.89646
